# Step 1: Tokenizer - 文本如何变成数字

## 学习目标

1. 理解 Tokenizer 的作用
2. **动手实现**字符级 Tokenizer 的 `encode` 和 `decode`
3. **动手实现** BPE 算法的核心步骤
4. 使用 SentencePiece 训练中文分词器

## 学习方式

1. 在这个 Notebook 中学习概念、运行代码、查看可视化
2. 去 `tokenizer_exercise.py` 完成 TODO 部分的代码
3. 回到这里验证你的实现
4. 卡住了？查看 `tokenizer_solution.py` 中的答案

---

## 1. 为什么需要 Tokenizer？

神经网络只能处理数字（张量），所以我们需要一种方式将文本转换为数字序列。

```
"Hello world" → Tokenizer → [15496, 995] → 神经网络 → [预测的token] → Tokenizer → "Hi"
```

关键要求：**这个映射必须是可逆的**（能从数字还原回文本）

---

## 2. 字符级 Tokenizer

最简单的方案：每个字符对应一个数字

### 2.1 理解词表 (Vocabulary)

In [ ]:
# 导入你的实现（完成 TODO 后运行）
from tokenizer_exercise import CharTokenizer

# 创建分词器
tokenizer = CharTokenizer()

# 查看初始词表（只有特殊token）
print("初始词表:")
print(tokenizer.vocab)

In [ ]:
# 从文本构建词表
text = "Hello world! 你好世界！"
tokenizer.build_vocab(text)

# 查看构建后的词表
print("\n构建后的词表:")
for char, idx in sorted(tokenizer.vocab.items(), key=lambda x: x[1]):
    print(f"  '{char}' -> {idx}")

### 2.2 练习：实现 encode 和 decode

现在去 `tokenizer_exercise.py`，完成 **TODO 1** 和 **TODO 2**：

- `encode(text)`: 将文本转换为 token ID 列表
- `decode(ids)`: 将 token ID 列表还原为文本

完成后，运行下面的代码验证：

In [ ]:
# 重新导入（如果你修改了代码）
import importlib
import tokenizer_exercise
importlib.reload(tokenizer_exercise)
from tokenizer_exercise import CharTokenizer

# 测试你的实现
tokenizer = CharTokenizer()
tokenizer.build_vocab("Hello world! 你好世界！")

test_text = "Hello 你好"
print(f"原文: {test_text}")

try:
    ids = tokenizer.encode(test_text)
    print(f"编码: {ids}")
    
    decoded = tokenizer.decode(ids)
    print(f"解码: {decoded}")
    
    if test_text == decoded:
        print("\n✅ 编码-解码一致，实现正确！")
    else:
        print(f"\n❌ 编码-解码不一致: '{test_text}' != '{decoded}'")
except NotImplementedError as e:
    print(f"\n⚠️ {e}")

### 2.3 可视化：字符到数字的映射

In [ ]:
# 可视化编码过程
def visualize_encoding(tokenizer, text):
    """可视化文本编码过程"""
    try:
        ids = tokenizer.encode(text)
        print(f"文本: {text}")
        print("      " + "   ".join([f"{c:^3}" for c in text]))
        print("       ↓   " * len(text))
        print("IDs:  " + "   ".join([f"{i:^3}" for i in ids]))
    except NotImplementedError:
        print("请先完成 encode 方法")

visualize_encoding(tokenizer, "Hello")

---

## 3. BPE (Byte Pair Encoding) 算法

### 3.1 为什么需要 BPE？

字符级分词的问题：
- 序列太长（"hello" → 5 个 token）
- 没有利用词的语义

词级分词的问题：
- 词表太大
- OOV（未登录词）问题

BPE 的解决方案：**子词级分词** —— 自动学习常见的字符组合

```
"unhappiness" → ["un", "happi", "ness"]  # 3 个有意义的子词
```

### 3.2 BPE 算法原理

核心思想：**反复合并最高频的相邻 token 对**

```
初始: l o w (出现5次), l o w e r (出现2次)

Step 1: 统计 pair 频率
  (l, o): 7次, (o, w): 7次, (w, e): 2次, (e, r): 2次

Step 2: 合并最高频的 (l, o) -> lo
  lo w (5次), lo w e r (2次)

Step 3: 继续统计和合并...
  (lo, w): 7次 -> low
  ...
```

### 3.3 练习：实现 BPE 统计函数

去 `tokenizer_exercise.py`，完成 **TODO 3**: `_get_stats` 方法

这个函数统计所有相邻 token 对的出现频率。

In [ ]:
# 重新导入
importlib.reload(tokenizer_exercise)
from tokenizer_exercise import BPETokenizer

# 测试 _get_stats
tokenizer = BPETokenizer()

# 输入：词列表，每个词是空格分隔的字符序列
words = [
    ("l o w", 5),      # "low" 出现 5 次
    ("l o w e r", 2),  # "lower" 出现 2 次
]

try:
    stats = tokenizer._get_stats(words)
    print("统计结果:")
    for pair, count in sorted(stats.items(), key=lambda x: -x[1]):
        print(f"  {pair}: {count}次")
    
    # 验证
    expected = {('l', 'o'): 7, ('o', 'w'): 7, ('w', 'e'): 2, ('e', 'r'): 2}
    if stats == expected:
        print("\n✅ _get_stats 实现正确！")
    else:
        print(f"\n❌ 结果不匹配")
        print(f"期望: {expected}")
except NotImplementedError as e:
    print(f"⚠️ {e}")

### 3.4 练习：完成 BPE 训练循环

去 `tokenizer_exercise.py`，完成 **TODO 4a, 4b, 4c**：

- 4a: 获取 pair 频率
- 4b: 找到频率最高的 pair
- 4c: 合并这个 pair

In [ ]:
# 重新导入
importlib.reload(tokenizer_exercise)
from tokenizer_exercise import BPETokenizer

# 训练 BPE
tokenizer = BPETokenizer()

text = """
low lower lowest lowly
new newer newest newly
show showed showing shown
the quick brown fox jumps over the lazy dog
the the the quick quick brown brown
""" * 5  # 重复以增加频率

try:
    tokenizer.train(text, vocab_size=80, verbose=True)
    print("\n✅ BPE 训练完成！")
except NotImplementedError as e:
    print(f"\n⚠️ {e}")

### 3.5 可视化：BPE 合并规则

In [ ]:
# 查看学到的合并规则
if hasattr(tokenizer, 'merges') and tokenizer.merges:
    print("学到的合并规则（前15个）:")
    for i, (pair, merged) in enumerate(list(tokenizer.merges.items())[:15]):
        print(f"  {i+1}. '{pair[0]}' + '{pair[1]}' -> '{merged}'")
else:
    print("还没有学到合并规则，请先完成训练")

In [ ]:
# 测试编码解码
if hasattr(tokenizer, 'merges') and tokenizer.merges:
    test_words = ["lower", "showing", "newest"]
    
    for word in test_words:
        ids = tokenizer.encode(word)
        tokens = [tokenizer.id_to_token[i] for i in ids]
        decoded = tokenizer.decode(ids)
        
        print(f"'{word}' -> {tokens} -> IDs: {ids} -> '{decoded}'")

---

## 4. 运行测试

完成所有 TODO 后，运行完整测试：

In [ ]:
# 运行测试文件
!python tokenizer_exercise.py

---

## 5. 对比：使用真实的 Tokenizer

让我们看看 GPT-2 的 tokenizer 是如何工作的：

In [ ]:
# 安装 tiktoken（如果需要）
# !pip install tiktoken

try:
    import tiktoken
    
    # GPT-2 使用的 tokenizer
    enc = tiktoken.get_encoding("gpt2")
    
    text = "Hello world! 你好世界！"
    ids = enc.encode(text)
    tokens = [enc.decode([i]) for i in ids]
    
    print(f"文本: {text}")
    print(f"Token 数量: {len(ids)}")
    print(f"Tokens: {tokens}")
    print(f"IDs: {ids}")
    print(f"\nGPT-2 词表大小: {enc.n_vocab}")
except ImportError:
    print("请安装 tiktoken: pip install tiktoken")

---

## 6. 验证清单

完成本步骤后，你应该能够：

- [ ] 解释 Tokenizer 的作用（文本 ↔ 数字）
- [ ] 实现 `encode` 方法（文本 → ID 列表）
- [ ] 实现 `decode` 方法（ID 列表 → 文本）
- [ ] 解释 BPE 算法（统计频率 → 合并高频 pair）
- [ ] 实现 BPE 的统计和训练循环

---

## 下一步

理解 Tokenizer 后，进入 [Step 2: GPT Model](../step2_gpt_model/) 学习如何构建处理这些 token 的模型。